In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler
import optuna
from optuna.trial import TrialState
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, train_test_split

# Fixed random seed for reproducibility
SEED = 42

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

def load_data():
    X_train = pd.read_csv('UCI_HAR_Dataset/train/X_train.txt', sep='\\s+', header=None)
    y_train = pd.read_csv('UCI_HAR_Dataset/train/y_train.txt', sep='\\s+', header=None, names=['Activity'])
    X_test = pd.read_csv('UCI_HAR_Dataset/test/X_test.txt', sep='\\s+', header=None)
    y_test = pd.read_csv('UCI_HAR_Dataset/test/y_test.txt', sep='\\s+', header=None, names=['Activity'])

    features = pd.read_csv('UCI_HAR_Dataset/features.txt', sep='\\s+', header=None)
    feature_names = features[1].tolist()
    
    X_train.columns = feature_names
    X_test.columns = feature_names

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data()

activity_labels = {
    1: 'WALKING',
    2: 'WALKING_UPSTAIRS',
    3: 'WALKING_DOWNSTAIRS',
    4: 'SITTING',
    5: 'STANDING',
    6: 'LAYING'
}

y_train['Activity'] = y_train['Activity'].map(activity_labels)
y_test['Activity'] = y_test['Activity'].map(activity_labels)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train['Activity'])
y_test_encoded = encoder.transform(y_test['Activity'])

X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train_encoded)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test_encoded)

class HAR_MLP(nn.Module):
    def __init__(self, input_size, num_classes, hidden_sizes, dropout_rates):
        super(HAR_MLP, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for i, (h_size, drop_rate) in enumerate(zip(hidden_sizes, dropout_rates)):
            layers.append(nn.Linear(prev_size, h_size))
            layers.append(nn.BatchNorm1d(h_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(drop_rate))
            prev_size = h_size
        
        layers.append(nn.Linear(prev_size, num_classes))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    return train_loss, train_acc

def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    
    eval_loss = running_loss / len(data_loader)
    eval_acc = correct / total
    return eval_loss, eval_acc, np.array(all_preds), np.array(all_targets)

def objective(trial):
    trial_seed = SEED + trial.number
    set_all_seeds(trial_seed)
    
    lr = trial.suggest_float('lr', 0.0001, 0.0004, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-7, 5e-6, log=True)
    
    n_layers = trial.suggest_int('n_layers', 2, 3)
    hidden_sizes = []
    dropout_rates = []
    
    for i in range(n_layers):
        hidden_sizes.append(trial.suggest_categorical(f'hidden_size_{i}', [64, 128, 256]))
        dropout_rates.append(trial.suggest_float(f'dropout_{i}', 0.1, 0.4))
    
    batch_size = trial.suggest_categorical('batch_size', [32, 64])
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    
    k_folds = 3
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=trial_seed)
    
    cv_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_tensor)):
        print(f"Trial {trial.number}, Fold {fold+1}/{k_folds}")
        
        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)
        
        generator = torch.Generator()
        generator.manual_seed(trial_seed + fold)
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=batch_size,
            sampler=train_sampler,
            worker_init_fn=lambda worker_id: np.random.seed(trial_seed + fold + worker_id)
        )
        
        val_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            sampler=val_sampler
        )
        
        model = HAR_MLP(
            input_size=X_train.shape[1],
            num_classes=len(np.unique(y_train_encoded)),
            hidden_sizes=hidden_sizes,
            dropout_rates=dropout_rates
        ).to(device)
        
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()
        
        best_val_acc = 0.0
        patience = 10
        patience_counter = 0
        max_epochs = 100
        
        for epoch in range(max_epochs):
            train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
            val_loss, val_acc, _, _ = evaluate(model, val_loader, criterion, device)
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break

            global_step = fold * max_epochs + epoch
            trial.report(val_acc, global_step)
            
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
        
        cv_scores.append(best_val_acc)
    
    mean_cv_score = np.mean(cv_scores)
    print(f"Trial {trial.number} completed with mean CV score: {mean_cv_score:.4f}")
    
    return mean_cv_score

def optimize_hyperparameters(n_trials=20):
    sampler = optuna.samplers.TPESampler(seed=SEED)
    
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5)
    study = optuna.create_study(
        direction='maximize',
        pruner=pruner,
        sampler=sampler
    )
    
    study.optimize(objective, n_trials=n_trials)
    
    print("\nResults:")
    print(f"  Completed trials: {len(study.trials)}")
    print(f"  Pruned trials: {len([t for t in study.trials if t.state == TrialState.PRUNED])}")
    
    print("\nBest trial:")
    trial = study.best_trial
    print(f"  Best accuracy: {trial.value:.4f}")
    print("  Parameters:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    
    return study, study.best_params

study, best_params = optimize_hyperparameters(n_trials=20)

Using device: cuda


[I 2025-05-13 20:46:32,593] A new study created in memory with name: no-name-0c21f2e7-6882-4644-8797-cda0214acac7


Trial 0, Fold 1/3
Trial 0, Fold 2/3
Trial 0, Fold 3/3


[I 2025-05-13 20:48:01,132] Trial 0 finished with value: 0.9870784935761331 and parameters: {'lr': 0.0001680720977739039, 'weight_decay': 4.123206532618725e-06, 'n_layers': 3, 'hidden_size_0': 64, 'dropout_0': 0.11742508365045984, 'hidden_size_1': 64, 'dropout_1': 0.10617534828874074, 'hidden_size_2': 64, 'dropout_2': 0.1545474901621302, 'batch_size': 64}. Best is trial 0 with value: 0.9870784935761331.


Trial 0 completed with mean CV score: 0.9871
Trial 1, Fold 1/3
Trial 1, Fold 2/3
Trial 1, Fold 3/3


...

Trial 18, Fold 1/3


[I 2025-05-13 20:56:12,532] Trial 18 pruned. 


Trial 19, Fold 1/3


[I 2025-05-13 20:56:16,065] Trial 19 pruned. 



Study statistics:
  Number of finished trials: 20
  Number of pruned trials: 14

Best trial:
  Value (Mean CV Accuracy): 0.9875
  Params:
    lr: 0.00023215521736070897
    weight_decay: 1.9485671251272545e-07
    n_layers: 2
    hidden_size_0: 128
    dropout_0: 0.19138413075201122
    hidden_size_1: 128
    dropout_1: 0.13661147045343366
    batch_size: 32
